# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [196]:
import pandas as pd
import requests
from functools import reduce

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [197]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'
r = requests.get(url)
data_json = r.json()

In [255]:
# 2) Seu código aqui
pvls_RJ = pd.DataFrame(data_json['items'])
lista_status = list(pvls_RJ['status'].unique())
qtde_status = list(map(lambda x: pvls_RJ[pvls_RJ['status']==x].shape[0], lista_status))
dict_status = dict(zip(lista_status, qtde_status))

def ordena_dict(dicionario):
    return dict(sorted(dicionario.items(), key=lambda t:t[1], reverse=True))
dict_status_decrescente = ordena_dict(dict_status)

print('O status mais frequente e a quantidade é: ', list(dict_status_decrescente.items())[0])
print('O segunda status mais frequente e a quantidade é: ', list(dict_status_decrescente.items())[1])
print('O terceiro status mais frequente e a quantidade é: ', list(dict_status_decrescente.items())[2])

O status mais frequente e a quantidade é:  ('Deferido', 95)
O segunda status mais frequente e a quantidade é:  ('Arquivado', 53)
O terceiro status mais frequente e a quantidade é:  ('Regularizado', 26)


In [199]:
# 3) Seu código aqui
pvls_RJ_copy = pvls_RJ.copy()
ano_status = pvls_RJ_copy['data_status'].str
pvls_RJ_copy['ano_status'] = ano_status[6:10]
lista_status_ano = pvls_RJ_copy['ano_status'].unique()
qtde_status_ano = list(map(lambda x: pvls_RJ_copy[pvls_RJ_copy['ano_status']==x].shape[0], lista_status_ano))
dict_status_ano = dict(zip(lista_status_ano, qtde_status_ano))
dict_status_ano

{'2008': 37,
 '2011': 10,
 '2016': 4,
 '2013': 15,
 '2020': 15,
 '2015': 8,
 '2007': 29,
 '2012': 13,
 '2006': 4,
 '2002': 4,
 '2009': 9,
 '2010': 19,
 '2014': 20,
 '2023': 9,
 '2022': 12,
 '2003': 1,
 '2004': 1,
 '2017': 5,
 '2019': 17,
 '2021': 5,
 '2018': 4}

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [246]:
#1) Seu código aqui
# Coletar dados por estados
url_estado = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?tipo_interessado=Estado'
r_estado = requests.get(url_estado)
data_json_estado = r_estado.json()
pvls_estados = pd.DataFrame(data_json_estado['items'])

# Coletar dados por municípios
url_municipio = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?tipo_interessado=Munic%C3%ADpio'
r_municipio = requests.get(url_municipio)
data_json_municipio = r_municipio.json()
pvls_municipios = pd.DataFrame(data_json_municipio['items'])
# https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?tipo_interessado=Munic%C3%ADpio

def consulta(tipo_interessado, uf):
    if (tipo_interessado=='Estado'):
        return pd.DataFrame(pvls_estados[pvls_estados['uf']==uf])
    elif (tipo_interessado=='Municipio'):
         return pd.DataFrame(pvls_municipios[pvls_municipios['uf']==uf])
    else:
        print('Precisa escolhar o tipo_interessado entre Estado e Municipio')

#consulta('Estado', 'BA')

In [247]:
# 2) Seu código aqui
dados_minas_gerais = consulta('Estado', 'MG')
qtde_arq_prazo = dados_minas_gerais[dados_minas_gerais['status']=='Arquivado por decurso de prazo'].shape[0]
print('Arquivado por decurso de prazo tem', qtde_arq_prazo, 'solicitação(ões).')

Arquivado por decurso de prazo tem 1 solicitação(ões).


In [254]:
# 3) Seu código aqui
dados_bahia = consulta('Municipio', 'BA')
dados_deferidos = dados_bahia[dados_bahia['status']=='Deferido']

lista_dados_deferidos = list(dados_deferidos['interessado'].unique())
qtde_dados_deferidos = list(map(lambda x: dados_deferidos[dados_deferidos['interessado']==x].shape[0], lista_dados_deferidos))
dict_dados_deferidos = dict(zip(lista_dados_deferidos, qtde_dados_deferidos))

def ordena_dict(dicionario):
    return dict(sorted(dicionario.items(), key=lambda t:t[1], reverse=True))
dict_dados_deferidos_decrescente = ordena_dict(dict_dados_deferidos)

print('O município da Bahia com mais solicitações deferidas é: ', list(dict_dados_deferidos_decrescente.items())[0])
#dict_dados_deferidos_decrescente

O município da Bahia com mais solicitações deferidas é:  ('Vitória da Conquista', 5)


In [259]:
# 4) Seu código aqui
dados_estado_bahia = consulta('Estado', 'BA')
dados_estado_bahia.to_csv('arquivo_dados_estado_BA.csv')